## Run the STEMMUS_SCOPE model
Steps to run the STEMMUS_SCOPE model, including preprocessing and postprocessing, on Surf super computer Snellius.

In [2]:
import os
import shutil
import numpy as np
from pathlib import Path
import subprocess
import datetime
import time

#### Create an executable file of STEMMUS_SCOPE

In [6]:
# path to executable
root_path_to_model = Path(Path.home(), "EcoExtreML", "STEMMUS_SCOPE")

Run this cell if:
- You are a developer of the model and you have already changed some of the MATLAB scripts.
- Make sure that you are on the right branch of STEMMUS_SCOPE repository.

Users on Snellius should be able to use MATLAB on Snellius and you must add the following command to [this bash script](https://github.com/EcoExtreML/processing/blob/main/run_jupyter_lab_on_compute_node.sh):<br>
`module load MATLAB/2021a-upd3`

In [2]:
# re-creating the exe file
result = subprocess.run(["mcc", "-m", "./src/STEMMUS_SCOPE_exe.m", "-a", "./src", "-d",
    "./exe", "-o", "STEMMUS_SCOPE", "-R", "nodisplay", "-R", "singleCompThread"],
    cwd = Path(root_path_to_model, "exe"))
result.check_returncode()

#### Update/set config files

In [10]:
# the user must provide the correct path
# path to config file
config_file_path = Path(root_path_to_model, "./config_file_snellius.txt")
# create an empty dict
config = {}
with open(config_file_path, "r") as f:
    for line in f:
        (key, val) = line.split("=")
        config[key] = val.rstrip('\n')

{'SoilPropertyPath': '/projects/0/einf2480/model_parameters/soil_property/',
 'InputPath': '/scratch-shared/ecoextreml/stemmus_scope/input/',
 'OutputPath': '/scratch-shared/ecoextreml/stemmus_scope/output/',
 'ForcingPath': '/projects/0/einf2480/forcing/plumber2_data/',
 'ForcingFileName': 'FI-Hyy_1996-2014_FLUXNET2015_Met.nc',
 'VegetationPropertyPath': '/projects/0/einf2480/model_parameters/vegetation_property/',
 'DurationSize': '17520'}

In [ ]:
# edit config
config["DurationSize"] = "20"

#### Create input directories, prepare input files 

In [4]:
def input_dir(station_name):
    """Create input directory and prepare input files
    """
    # get start time with the format Y-M-D-HM
    timestamp = time.strftime('%Y%m%d_%H%M')
    # create input directory
    work_dir = Path(config["InputPath"], station_name + '_' + timestamp)
    Path(work_dir).mkdir(parents=True, exist_ok=True)
    print(f"Prepare work directory {work_dir} for the station: {station_name}")
    # copy model parameters to work directory
    shutil.copytree(config["VegetationPropertyPath"], work_dir, dirs_exist_ok=True)
    # update config file for ForcingFileName and InputPath
    config_file_path = Path(work_dir, f"{station_name}_{timestamp}_config.txt")
    with open(config_file_path, 'w') as f:
        for i in config.keys():
            if i == "ForcingFileName":
                f.write(i + "=" + ncfile.stem + ".nc" + "\n")
            elif i == "InputPath":
                f.write(i + "=" + str(work_dir) + "/" + "\n")
            else:
                f.write(i + "=" + config[i] + "\n")

    return work_dir, config_file_path
    

In [5]:
# specify the forcing filenames
# forcing_filenames_list is by default none, if full_run is true, then all stations with forcing
# listed in the ForcingPath will be used.
forcing_filenames_list = ["NL-Hor_2008-2011_FLUXNET2015_Met.nc",
 "ZA-Kru_2000-2002_FLUXNET2015_Met.nc"]

full_run = False
if full_run:
    forcing_filenames_list = Path(config["ForcingPath"]).iterdir()

config_path_dict = {}
work_dir_dict = {}
for ncfile in forcing_filenames_list:
    station_name = ncfile.stem.split('_')[0]
    work_dir_dict[station_name], config_path_dict[station_name] = input_dir(station_name)

Prepare work directory for the station: NL-Hor
Prepare work directory for the station: ZA-Kru


#### Run the model

In [6]:
# generate a text file
root_path = Path().resolve().parent
str.split("/")[-1]
for ncfile in forcing_filenames_list:
    station_name = ncfile.stem.split("_")[0]
    path_to_config = config_path_dict[station_name]
    # set matlab log dir to slurm, otherwise java.log files are created in user home dir
    os.environ['MATLAB_LOG_DIR'] = work_dir_dict[station_name]
    # run the model
    print(f"Run STEMMUS-SCOPE for the station: {station_name}")
    result = subprocess.run([f"exe/STEMMUS_SCOPE {path_to_config}"], cwd = root_path_to_model, shell=True)
    result.check_returncode()

Opening log file:  /gpfs/home3/yangl/processing/out/java.log.8605
Reading config from /scratch-shared/ecoextreml/stemmus_scope/test/input/ZA-Kru_2022-05-10-1135/ZA-Kru_2022-05-10-1135_config.txt

 The calculations start now 
Opening log file:  /gpfs/home3/yangl/processing/out/java.log.9641
Reading config from /scratch-shared/ecoextreml/stemmus_scope/test/input/NL-Hor_2022-05-10-1135/NL-Hor_2022-05-10-1135_config.txt

 The calculations start now 


#### Create output directories, prepare output files

In [7]:
# convert csv files to nc files
path_to_utils = Path(root_path_to_model, "utils/csv_to_nc")
for ncfile in forcing_filenames_list:
    station_name = ncfile.stem.split("_")[0]
    path_to_config = config_path_dict[station_name]
    result = subprocess.run(["python", Path(path_to_utils, "generate_netcdf_files.py"),
         "--config_file", path_to_config, "--variable_file",
         Path(path_to_utils, "Variables_will_be_in_NetCDF_file.csv")])
    result.check_returncode()

/scratch-shared/ecoextreml/stemmus_scope/test/output/ZA-Kru_2022-05-10-1135/ECdata.csv
Reading variable metadata from /home/yangl/STEMMUS_SCOPE/utils/csv_to_nc/Variables_will_be_in_NetCDF_file.csv
Creating /scratch-shared/ecoextreml/stemmus_scope/test/output/ZA-Kru_2022-05-10-1135/ZA-Kru_2022-05-10-1135_STEMMUS_SCOPE.nc 
Reading data from file: radiation.csv
Reading data from file: fluxes.csv
Reading data from file: surftemp.csv
Reading data from file: Sim_Temp.csv
Reading data from file: Sim_Theta.csv
Reading data from file: aerodyn.csv
Reading data from file: ECdata.csv
Done writing /scratch-shared/ecoextreml/stemmus_scope/test/output/ZA-Kru_2022-05-10-1135/ZA-Kru_2022-05-10-1135_STEMMUS_SCOPE.nc
/scratch-shared/ecoextreml/stemmus_scope/test/output/NL-Hor_2022-05-10-1136/ECdata.csv
Reading variable metadata from /home/yangl/STEMMUS_SCOPE/utils/csv_to_nc/Variables_will_be_in_NetCDF_file.csv
Creating /scratch-shared/ecoextreml/stemmus_scope/test/output/NL-Hor_2022-05-10-1136/NL-Hor_202